<a href="https://colab.research.google.com/github/antonionipo/Modelos-de-Classifica-o-com-Python/blob/main/Otimiza%C3%A7%C3%A3o%20de%20Hiperpar%C3%A2metros/Otimizacao_de_Hiperparametros_exercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Após prever o preço das casas, vocês implantarão o GridSearch no modelo de GradientBoosting para tentar melhorar a performance do modelo, aumentando o R^2 e diminuindo o MAE.

In [94]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [95]:
df = pd.read_excel('house.xlsx')
df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,Gd,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,TA,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,Gd,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,TA,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,Gd,...,192,84,0,0,0,0,0,12,2008,250000


In [96]:
df.dropna(inplace=True)
df.isnull().sum()

,0
Id,0
MSSubClass,0
LotFrontage,0
LotArea,0
OverallQual,0
OverallCond,0
YearBuilt,0
YearRemodAdd,0
MasVnrArea,0
ExterQual,0


In [97]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1121.000000,1121.000000,1121.000000,1121.000000,1121.000000,1121.000000,1121.000000,1121.000000,1121.000000,1121.000000,...,1121.000000,1121.000000,1121.000000,1121.000000,1121.000000,1121.000000,1121.000000,1121.000000,1121.000000,1121.000000
mean,726.842105,56.146298,70.665477,10122.956289,6.212310,5.561106,1972.264942,1985.683318,108.487957,438.399643,...,92.610169,46.130241,21.844781,3.348796,16.100803,2.934880,23.429081,6.339875,2007.795718,185506.152542
std,420.769012,41.772684,24.266812,8129.302641,1.380822,1.068236,30.967730,21.025974,189.500966,468.385038,...,121.868623,64.390029,61.282083,29.772540,57.801829,40.222087,165.759816,2.687854,1.335043,82999.159004
min,1.000000,20.000000,21.000000,1300.000000,2.000000,2.000000,1880.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,35311.000000
25%,364.000000,20.000000,60.000000,7590.000000,5.000000,5.000000,1953.000000,1966.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,131000.000000
50%,725.000000,50.000000,70.000000,9416.000000,6.000000,5.000000,1974.000000,1995.000000,0.000000,374.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,164900.000000
75%,1091.000000,70.000000,80.000000,11361.000000,7.000000,6.000000,2003.000000,2005.000000,170.000000,702.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,219500.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,648.000000,2500.000000,12.000000,2010.000000,755000.000000


In [98]:
features = pd.get_dummies(df)
features.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,YrSold,SalePrice,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_TA
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,2008,208500,False,False,True,False,False,False,False,True
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,2007,181500,False,False,False,True,False,False,False,True
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,2008,223500,False,False,True,False,False,False,False,True
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,2006,140000,False,False,False,True,False,False,False,True
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,2008,250000,False,False,True,False,False,False,False,True


In [99]:
labels = np.array(df['SalePrice'])
features = features.drop('SalePrice', axis = 1)
feature_list = list(features.columns)
features = np.array(features)

In [100]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [103]:
gbr = GradientBoostingRegressor(n_estimators = 100)
gbr.fit(train_features, train_labels)
gbr_pred = gbr.predict(test_features)

In [108]:
parameters = {'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05],
                  'subsample'    : [0.2],
                  'n_estimators' : [50, 100, 500, 1000, 1500, 20000],
                  'max_depth'    : [4, 8, 15, 25]
                 }
grid_search = GridSearchCV(gbr, parameters, scoring='r2', cv=2, n_jobs=-1)
grid_search.fit(train_features, train_labels)

In [110]:
best_model = grid_search.best_estimator_
parameters = best_model.get_params()

In [112]:
grb_tunned = GradientBoostingRegressor(**parameters)
grb_tunned.fit(train_features, train_labels)
gbr_tunned_pred = grb_tunned.predict(test_features)

In [114]:
# Calculate the absolute errors
errors_gbr = abs(gbr_pred - test_labels)
errors_gbr_tunned = abs(gbr_tunned_pred - test_labels)

# Print out the mean absolute error (mae)
r_sq = gbr.score(test_features, test_labels)
print('Coeficiente de Determinação (R²):', r_sq)
r_sq = grb_tunned.score(test_features, test_labels)
print('Coeficiente de Determinação (R²):', r_sq)
print('MAE:', metrics.mean_absolute_error(test_labels, gbr_pred))
print('MAE_tunned:', metrics.mean_absolute_error(test_labels, gbr_tunned_pred))

Coeficiente de Determinação (R²): 0.8943607112738183
Coeficiente de Determinação (R²): 0.8810569910048135
MAE: 18563.669659518502
MAE_tunned: 19099.47213326251
